<a href="https://colab.research.google.com/github/squareleaf/mission_wired_data/blob/main/MissionWired_DataEngineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import packages**

In [1]:
import pandas as pd

**Import data**

In [16]:
con_info = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons.csv')
con_emails = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email.csv')
con_sub_status = pd.read_csv('https://als-hiring.s3.amazonaws.com/fake_data/2020-07-01_17%3A11%3A00/cons_email_chapter_subscription.csv')

**Transform data**

*For constituent subscription status, strip out any rows where chapter_id is not 1. Includes some basic checking to see how much was removed.*

In [17]:
print('Pre-transform')
con_sub_status.describe()

Pre-transform


,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub
count,350000.000000,350000.000000,350000.000000,350000.000000
mean,175000.500000,350695.017049,1.425889,0.901354
std,101036.441446,201952.593769,0.901675,0.298186
min,1.000000,3.000000,1.000000,0.000000
25%,87500.750000,176301.500000,1.000000,1.000000
50%,175000.500000,351203.000000,1.000000,1.000000
75%,262500.250000,525782.250000,1.000000,1.000000
max,350000.000000,699997.000000,4.000000,1.000000


In [18]:
con_sub_status_clean = con_sub_status[con_sub_status.chapter_id == 1]
print('Post transform')
con_sub_status_clean.describe()

Post transform


,cons_email_chapter_subscription_id,cons_email_id,chapter_id,isunsub
count,275484.00000,275484.000000,275484.0,275484.000000
mean,160546.33146,350641.249456,1.0,0.901192
std,100433.82361,201968.490979,0.0,0.298404
min,1.00000,3.000000,1.0,0.000000
25%,72476.50000,176056.500000,1.0,1.000000
50%,153484.50000,351257.500000,1.0,1.000000
75%,244934.25000,525690.750000,1.0,1.000000
max,349999.00000,699997.000000,1.0,1.000000


**Create people file**

*Produce a people file by combining data from all three dataframes*

First, determine which dataframes have the data we need

In [23]:
print('Info')
con_info.columns

Info


Index(['cons_id', 'prefix', 'firstname', 'middlename', 'lastname', 'suffix',
       'salutation', 'gender', 'birth_dt', 'title', 'employer', 'occupation',
       'income', 'source', 'subsource', 'userid', 'password', 'is_validated',
       'is_banned', 'change_password_next_login', 'consent_type_id',
       'create_dt', 'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')

In [24]:
print('\n Emails')
con_emails.columns


 Emails


Index(['cons_email_id', 'cons_id', 'cons_email_type_id', 'is_primary', 'email',
       'canonical_local_part', 'domain', 'double_validation', 'create_dt',
       'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note'],
      dtype='object')

In [25]:
print('\n Status')
con_sub_status_clean.columns


 Status


Index(['cons_email_chapter_subscription_id', 'cons_email_id', 'chapter_id',
       'isunsub', 'unsub_dt', 'modified_dt'],
      dtype='object')

Before merging the con_info and con_emails dataframes, simplify the con_emails dataframe to only include necessary columns. Then, filter to only email addresses marked as primary. If the dataframes are ever very large, this should make subsequent steps faster.

In [31]:
con_emails = con_emails[['cons_email_id', 'cons_id', 'is_primary', 'email']]
primary_emails = con_emails[con_emails.is_primary == 1]
primary_emails.head(n=2)

,cons_email_id,cons_id,is_primary,email
0,1,548198,1,xmartinez@vincent.com
1,2,491137,1,hmiller@haynes.biz


Also, simplify the status dataframe to only those columns that will be needed to join or for data

In [33]:
con_sub_status_clean = con_sub_status_clean[['cons_email_id', 'isunsub']]
con_sub_status_clean.head(n=2)

,cons_email_id,isunsub
0,332188,1
1,536526,1


Join and merge the con_info and con_emails dataframes, since they both have the cons_id

In [32]:
con_info_and_emails = pd.merge(con_info, primary_emails, how='left', on='cons_id')
con_info_and_emails.columns

Index(['cons_id', 'prefix', 'firstname', 'middlename', 'lastname', 'suffix',
       'salutation', 'gender', 'birth_dt', 'title', 'employer', 'occupation',
       'income', 'source', 'subsource', 'userid', 'password', 'is_validated',
       'is_banned', 'change_password_next_login', 'consent_type_id',
       'create_dt', 'create_app', 'create_user', 'modified_dt', 'modified_app',
       'modified_user', 'status', 'note', 'cons_email_id', 'is_primary',
       'email'],
      dtype='object')

Join the resulting dataframe with the status dataframe and keep only necessary columns

In [36]:
people = pd.merge(con_info_and_emails, con_sub_status_clean, how='left', on='cons_email_id')
people = people[['email', 'source', 'isunsub', 'create_dt', 'modified_dt']]
people.head(n=2)

,email,source,isunsub,create_dt,modified_dt
0,daniel72@hudson.com,google,NaN,"Fri, 1983-08-26 06:02:03","Sun, 2015-12-27 09:28:02"
1,caustin@spears-carson.com,facebook,1.0,"Mon, 1979-03-05 21:08:54","Tue, 1989-06-20 13:28:57"


Rename the columns to the requested format and output as csv

In [38]:
people.rename(columns={'source': 'code', 'is_unsub': 'isunsub', 'create_dt': 'created_dt', 'modified_dt': 'updated_dt'}, inplace=True)
people.head(n=2)

people.to_csv('people.csv', header=True, index=False)

**Create acquisition facts file**

*Using the people file, create a file about when constituents were acquired*